In [32]:
#imports to work with...
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from torch.utils.data import DataLoader
import torch
import torchvision
from torchvision import transforms
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import label_binarize

from cl_framework.continual_learning.metrics.metric_evaluator_incdec import MetricEvaluatorIncDec
from cl_framework.utilities.matrix_logger import IncDecLogger
from torchmetrics import Recall

In [33]:
subcategories_names = ['eating burger','eating cake','eating carrots','eating chips','eating doughnuts',
                       'eating hotdog','eating ice cream','eating spaghetti','eating watermelon','sucking lolly',
                       'tasting beer','tasting food','tasting wine','sipping cup','texting','talking on cell phone',
                       'looking at phone','smoking','smoking hookah','smoking pipe','sleeping','yawning','headbanging',
                       'headbutting','shaking head','scrubbing face','putting in contact lenses','putting on eyeliner',
                       'putting on foundation','putting on lipstick','putting on mascara','brushing hair','brushing teeth',
                       'braiding hair','combing hair','dyeing eyebrows','dyeing hair']
all_behaviors_dict = {
    'food': [
        'eating burger', 'eating cake', 'eating carrots', 'eating chips', 'eating doughnuts',
        'eating hotdog', 'eating ice cream', 'eating spaghetti', 'eating watermelon',
        'sucking lolly', 'tasting beer', 'tasting food', 'tasting wine', 'sipping cup'
    ],
    'phone': [
        'texting', 'talking on cell phone', 'looking at phone'
    ],
    'smoking': [
        'smoking', 'smoking hookah', 'smoking pipe'
    ],
    'fatigue': [
        'sleeping', 'yawning', 'headbanging', 'headbutting', 'shaking head'
    ],
    'selfcare': [
        'scrubbing face', 'putting in contact lenses', 'putting on eyeliner', 'putting on foundation',
        'putting on lipstick', 'putting on mascara', 'brushing hair', 'brushing teeth', 'braiding hair',
        'combing hair', 'dyeing eyebrows', 'dyeing hair'
    ]
}

In [34]:
# ap of subcategories
mean_path = 'mean_over_tasks/forg_ap_per_subcategory.out'
std_path = 'std_over_tasks/forg_ap_per_subcategory.out'

path_to_baseline = '../runs_trainings/no_freeze/multilabel/weighted/statistics/'
baseline_data = np.loadtxt(os.path.join(path_to_baseline,mean_path),delimiter=',')
baseline_std_data = np.loadtxt(os.path.join(path_to_baseline,std_path),delimiter=',')

path_to_decremental = '../runs_trainings/no_freeze/decremental_multilabel/weighted/statistics/'
decremental_data = np.loadtxt(os.path.join(path_to_decremental,mean_path),delimiter=',')
decremental_std_data = np.loadtxt(os.path.join(path_to_decremental,std_path),delimiter=',')

path_to_incdec = '../runs_trainings/no_freeze/incremental_decremental_multilabel/weighted/statistics/'
incdec_data = np.loadtxt(os.path.join(path_to_incdec,mean_path),delimiter=',')
incdec_std_data = np.loadtxt(os.path.join(path_to_incdec,std_path),delimiter=',')

path_to_joint = '../runs_trainings/no_freeze/joint_incremental_multilabel/weighted/statistics/'
joint_data = np.loadtxt(os.path.join(path_to_joint,mean_path),delimiter=',')
joint_std_data = np.loadtxt(os.path.join(path_to_joint,std_path),delimiter=',')

In [35]:
output_path = '../statistics_to_save/subcategories_plots/'
if not os.path.exists(output_path):
    os.mkdir(output_path)
ouput_ap_path = os.path.join(output_path,'ap')
if not os.path.exists(ouput_ap_path):
    os.mkdir(ouput_ap_path)
no_freeze_path = os.path.join(ouput_ap_path,'no_freeze')
if not os.path.exists(no_freeze_path):
    os.mkdir(no_freeze_path)

for class_name in all_behaviors_dict:
    subcat_names = all_behaviors_dict[class_name]

    baseline_list = []
    decremental_list = []
    incdec_list = []
    joint_list = []
    for idx_subcat in range(len(subcat_names)):
        pos_subcat_in_data = subcategories_names.index(subcat_names[idx_subcat])
        baseline_list.append(baseline_data[pos_subcat_in_data])
        decremental_list.append(decremental_data[pos_subcat_in_data])
        incdec_list.append(incdec_data[pos_subcat_in_data])
        joint_list.append(joint_data[pos_subcat_in_data])
        

    baseline_std_list = []
    decremental_std_list = []
    incdec_std_list = []
    joint_std_list = []
    for idx_subcat in range(len(subcat_names)):
        pos_subcat_in_data = subcategories_names.index(subcat_names[idx_subcat])
        baseline_std_list.append(baseline_std_data[pos_subcat_in_data])
        decremental_std_list.append(decremental_std_data[pos_subcat_in_data])
        incdec_std_list.append(incdec_std_data[pos_subcat_in_data])
        joint_std_list.append(joint_std_data[pos_subcat_in_data])
        

    subcat_means = {
        'Baseline': [baseline_list,baseline_std_list],
        'Decremental': [decremental_list,decremental_std_list],
        'Incdec': [incdec_list,incdec_std_list],
        'Joint': [joint_list,joint_std_list],
        }

    x = np.arange(len(subcat_names))  # the label locations
    width = 0.20  # the width of the bars
    multiplier = 0

    fig, ax = plt.subplots(layout='constrained')

    for attribute, measurements in subcat_means.items():
        offset = width * multiplier
        rects = ax.bar(x + offset, measurements[0], width, label=attribute)
        ax.errorbar(x+offset, measurements[0], measurements[1], fmt='.', color='Black', elinewidth=1,capthick=5,errorevery=1, alpha=0.5, ms=4, capsize = 2)
        #ax.bar_label(rects, padding=3)
        multiplier += 1

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Forgetting on subcat AP')
    ax.set_title('Subcategories Forgetting AP - {}'.format(class_name))
    ax.set_xticks(x + width, subcat_names)
    ax.legend(loc='upper left', ncols=4)
    #ax.set_ylim(-5, 15)
    ax.tick_params(axis='x', labelrotation=90)
    ax.axhline(linewidth=1, color='black')
    fig.savefig(no_freeze_path + '/class_' + class_name + '.png')
    plt.close(fig)

In [36]:
# recall of subcategories
mean_path = 'mean_over_tasks/forg_recall_per_subcategory.out'
std_path = 'std_over_tasks/forg_recall_per_subcategory.out'

path_to_baseline = '../runs_trainings/no_freeze/multilabel/weighted/statistics/'
baseline_data = np.loadtxt(os.path.join(path_to_baseline,mean_path),delimiter=',')
baseline_std_data = np.loadtxt(os.path.join(path_to_baseline,std_path),delimiter=',')

path_to_decremental = '../runs_trainings/no_freeze/decremental_multilabel/weighted/statistics/'
decremental_data = np.loadtxt(os.path.join(path_to_decremental,mean_path),delimiter=',')
decremental_std_data = np.loadtxt(os.path.join(path_to_decremental,std_path),delimiter=',')

path_to_incdec = '../runs_trainings/no_freeze/incremental_decremental_multilabel/weighted/statistics/'
incdec_data = np.loadtxt(os.path.join(path_to_incdec,mean_path),delimiter=',')
incdec_std_data = np.loadtxt(os.path.join(path_to_incdec,std_path),delimiter=',')

path_to_joint = '../runs_trainings/no_freeze/joint_incremental_multilabel/weighted/statistics/'
joint_data = np.loadtxt(os.path.join(path_to_joint,mean_path),delimiter=',')
joint_std_data = np.loadtxt(os.path.join(path_to_joint,std_path),delimiter=',')

In [37]:
# recall
output_path = '../statistics_to_save/subcategories_plots/'
if not os.path.exists(output_path):
    os.mkdir(output_path)
ouput_ap_path = os.path.join(output_path,'recall')
if not os.path.exists(ouput_ap_path):
    os.mkdir(ouput_ap_path)
no_freeze_path = os.path.join(ouput_ap_path,'no_freeze')
if not os.path.exists(no_freeze_path):
    os.mkdir(no_freeze_path)

for class_name in all_behaviors_dict:
    subcat_names = all_behaviors_dict[class_name]

    baseline_list = []
    decremental_list = []
    incdec_list = []
    joint_list = []
    for idx_subcat in range(len(subcat_names)):
        pos_subcat_in_data = subcategories_names.index(subcat_names[idx_subcat])
        baseline_list.append(baseline_data[pos_subcat_in_data])
        decremental_list.append(decremental_data[pos_subcat_in_data])
        incdec_list.append(incdec_data[pos_subcat_in_data])
        joint_list.append(joint_data[pos_subcat_in_data])

    baseline_std_list = []
    decremental_std_list = []
    incdec_std_list = []
    joint_std_list = []
    for idx_subcat in range(len(subcat_names)):
        pos_subcat_in_data = subcategories_names.index(subcat_names[idx_subcat])
        baseline_std_list.append(baseline_std_data[pos_subcat_in_data])
        decremental_std_list.append(decremental_std_data[pos_subcat_in_data])
        incdec_std_list.append(incdec_std_data[pos_subcat_in_data])
        joint_std_list.append(joint_std_data[pos_subcat_in_data])
        

    subcat_means = {
        'Baseline': [baseline_list,baseline_std_list],
        'Decremental': [decremental_list,decremental_std_list],
        'Incdec': [incdec_list,incdec_std_list],
        'Joint': [joint_list,joint_std_list],
        }

    x = np.arange(len(subcat_names))  # the label locations
    width = 0.20  # the width of the bars
    multiplier = 0

    fig, ax = plt.subplots(layout='constrained')

    for attribute, measurements in subcat_means.items():
        offset = width * multiplier
        rects = ax.bar(x + offset, measurements[0], width, label=attribute)
        ax.errorbar(x+offset, measurements[0], measurements[1], fmt='.', color='Black', elinewidth=1,capthick=5,errorevery=1, alpha=0.5, ms=4, capsize = 2)
        #ax.bar_label(rects, padding=3)
        multiplier += 1

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Forgetting on subcat Recall')
    ax.set_title('Subcategories Forgetting Recall - {}'.format(class_name))
    ax.set_xticks(x + width, subcat_names)
    ax.legend(loc='upper left', ncols=4)
    #ax.set_ylim(-5, 15)
    ax.tick_params(axis='x', labelrotation=90)
    ax.axhline(linewidth=1, color='black')
    fig.savefig(no_freeze_path + '/class_' + class_name + '.png')
    plt.close(fig)

In [38]:
# precision of subcategories
mean_path = 'mean_over_tasks/forg_precision_per_subcategory.out'
std_path = 'std_over_tasks/forg_precision_per_subcategory.out'

path_to_baseline = '../runs_trainings/no_freeze/multilabel/weighted/statistics/'
baseline_data = np.loadtxt(os.path.join(path_to_baseline,mean_path),delimiter=',')
baseline_std_data = np.loadtxt(os.path.join(path_to_baseline,std_path),delimiter=',')

path_to_decremental = '../runs_trainings/no_freeze/decremental_multilabel/weighted/statistics/'
decremental_data = np.loadtxt(os.path.join(path_to_decremental,mean_path),delimiter=',')
decremental_std_data = np.loadtxt(os.path.join(path_to_decremental,std_path),delimiter=',')

path_to_incdec = '../runs_trainings/no_freeze/incremental_decremental_multilabel/weighted/statistics/'
incdec_data = np.loadtxt(os.path.join(path_to_incdec,mean_path),delimiter=',')
incdec_std_data = np.loadtxt(os.path.join(path_to_incdec,std_path),delimiter=',')

path_to_joint = '../runs_trainings/no_freeze/joint_incremental_multilabel/weighted/statistics/'
joint_data = np.loadtxt(os.path.join(path_to_joint,mean_path),delimiter=',')
joint_std_data = np.loadtxt(os.path.join(path_to_joint,std_path),delimiter=',')

In [39]:
# precision
output_path = '../statistics_to_save/subcategories_plots/'
if not os.path.exists(output_path):
    os.mkdir(output_path)
ouput_ap_path = os.path.join(output_path,'precision')
if not os.path.exists(ouput_ap_path):
    os.mkdir(ouput_ap_path)
no_freeze_path = os.path.join(ouput_ap_path,'no_freeze')
if not os.path.exists(no_freeze_path):
    os.mkdir(no_freeze_path)

for class_name in all_behaviors_dict:
    subcat_names = all_behaviors_dict[class_name]

    baseline_list = []
    decremental_list = []
    incdec_list = []
    joint_list = []
    for idx_subcat in range(len(subcat_names)):
        pos_subcat_in_data = subcategories_names.index(subcat_names[idx_subcat])
        baseline_list.append(baseline_data[pos_subcat_in_data])
        decremental_list.append(decremental_data[pos_subcat_in_data])
        incdec_list.append(incdec_data[pos_subcat_in_data])
        joint_list.append(joint_data[pos_subcat_in_data])
        

    baseline_std_list = []
    decremental_std_list = []
    incdec_std_list = []
    joint_std_list = []
    for idx_subcat in range(len(subcat_names)):
        pos_subcat_in_data = subcategories_names.index(subcat_names[idx_subcat])
        baseline_std_list.append(baseline_std_data[pos_subcat_in_data])
        decremental_std_list.append(decremental_std_data[pos_subcat_in_data])
        incdec_std_list.append(incdec_std_data[pos_subcat_in_data])
        joint_std_list.append(joint_std_data[pos_subcat_in_data])
        

    subcat_means = {
        'Baseline': [baseline_list,baseline_std_list],
        'Decremental': [decremental_list,decremental_std_list],
        'Incdec': [incdec_list,incdec_std_list],
        'Joint': [joint_list,joint_std_list],
        }

    x = np.arange(len(subcat_names))  # the label locations
    width = 0.20  # the width of the bars
    multiplier = 0

    fig, ax = plt.subplots(layout='constrained')

    for attribute, measurements in subcat_means.items():
        offset = width * multiplier
        rects = ax.bar(x + offset, measurements[0], width, label=attribute)
        ax.errorbar(x+offset, measurements[0], measurements[1], fmt='.', color='Black', elinewidth=1,capthick=5,errorevery=1, alpha=0.5, ms=4, capsize = 2)
        #ax.bar_label(rects, padding=3)
        multiplier += 1

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Forgetting on subcat Precision')
    ax.set_title('Subcategories Forgetting Precision - {}'.format(class_name))
    ax.set_xticks(x + width, subcat_names)
    ax.legend(loc='upper left', ncols=4)
    #ax.set_ylim(-5, 15)
    ax.tick_params(axis='x', labelrotation=90)
    ax.axhline(linewidth=1, color='black')
    fig.savefig(no_freeze_path + '/class_' + class_name + '.png')
    plt.close(fig)

In [ ]:
# accuracy of subcategories
mean_path = 'mean_over_tasks/forg_accuracy_per_subcategory.out'
std_path = 'std_over_tasks/forg_accuracy_per_subcategory.out'

path_to_baseline = '../runs_trainings/no_freeze/multilabel/weighted/statistics/'
baseline_data = np.loadtxt(os.path.join(path_to_baseline,mean_path),delimiter=',')
baseline_std_data = np.loadtxt(os.path.join(path_to_baseline,std_path),delimiter=',')

path_to_decremental = '../runs_trainings/no_freeze/decremental_multilabel/weighted/statistics/'
decremental_data = np.loadtxt(os.path.join(path_to_decremental,mean_path),delimiter=',')
decremental_std_data = np.loadtxt(os.path.join(path_to_decremental,std_path),delimiter=',')

path_to_incdec = '../runs_trainings/no_freeze/incremental_decremental_multilabel/weighted/statistics/'
incdec_data = np.loadtxt(os.path.join(path_to_incdec,mean_path),delimiter=',')
incdec_std_data = np.loadtxt(os.path.join(path_to_incdec,std_path),delimiter=',')

path_to_joint = '../runs_trainings/no_freeze/joint_incremental_multilabel/weighted/statistics/'
joint_data = np.loadtxt(os.path.join(path_to_joint,mean_path),delimiter=',')
joint_std_data = np.loadtxt(os.path.join(path_to_joint,std_path),delimiter=',')

In [ ]:
# precision
output_path = '../statistics_to_save/subcategories_plots/'
if not os.path.exists(output_path):
    os.mkdir(output_path)
ouput_ap_path = os.path.join(output_path,'accuracy')
if not os.path.exists(ouput_ap_path):
    os.mkdir(ouput_ap_path)
no_freeze_path = os.path.join(ouput_ap_path,'no_freeze')
if not os.path.exists(no_freeze_path):
    os.mkdir(no_freeze_path)

for class_name in all_behaviors_dict:
    subcat_names = all_behaviors_dict[class_name]

    baseline_list = []
    decremental_list = []
    incdec_list = []
    joint_list = []
    for idx_subcat in range(len(subcat_names)):
        pos_subcat_in_data = subcategories_names.index(subcat_names[idx_subcat])
        baseline_list.append(baseline_data[pos_subcat_in_data])
        decremental_list.append(decremental_data[pos_subcat_in_data])
        incdec_list.append(incdec_data[pos_subcat_in_data])
        joint_list.append(joint_data[pos_subcat_in_data])
        

    baseline_std_list = []
    decremental_std_list = []
    incdec_std_list = []
    joint_std_list = []
    for idx_subcat in range(len(subcat_names)):
        pos_subcat_in_data = subcategories_names.index(subcat_names[idx_subcat])
        baseline_std_list.append(baseline_std_data[pos_subcat_in_data])
        decremental_std_list.append(decremental_std_data[pos_subcat_in_data])
        incdec_std_list.append(incdec_std_data[pos_subcat_in_data])
        joint_std_list.append(joint_std_data[pos_subcat_in_data])
        

    subcat_means = {
        'Baseline': [baseline_list,baseline_std_list],
        'Decremental': [decremental_list,decremental_std_list],
        'Incdec': [incdec_list,incdec_std_list],
        'Joint': [joint_list,joint_std_list],
        }

    x = np.arange(len(subcat_names))  # the label locations
    width = 0.20  # the width of the bars
    multiplier = 0

    fig, ax = plt.subplots(layout='constrained')

    for attribute, measurements in subcat_means.items():
        offset = width * multiplier
        rects = ax.bar(x + offset, measurements[0], width, label=attribute)
        ax.errorbar(x+offset, measurements[0], measurements[1], fmt='.', color='Black', elinewidth=1,capthick=5,errorevery=1, alpha=0.5, ms=4, capsize = 2)
        #ax.bar_label(rects, padding=3)
        multiplier += 1

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Forgetting on subcat Accuracy')
    ax.set_title('Subcategories Accuracy Precision - {}'.format(class_name))
    ax.set_xticks(x + width, subcat_names)
    ax.legend(loc='upper left', ncols=4)
    #ax.set_ylim(-5, 15)
    ax.tick_params(axis='x', labelrotation=90)
    ax.axhline(linewidth=1, color='black')
    fig.savefig(no_freeze_path + '/class_' + class_name + '.png')
    plt.close(fig)

FROM HERE ONLY FROZEN MODELS

In [40]:
# ap of subcategories
mean_path = 'mean_over_tasks/forg_ap_per_subcategory.out'
std_path = 'std_over_tasks/forg_ap_per_subcategory.out'

path_to_baseline = '../runs_trainings/freeze_backbone/baseline_multilabel/weighted/new/statistics/'
baseline_data = np.loadtxt(os.path.join(path_to_baseline,mean_path),delimiter=',')
baseline_std_data = np.loadtxt(os.path.join(path_to_baseline,std_path),delimiter=',')

path_to_decremental = '../runs_trainings/freeze_backbone/decremental_multilabel/weighted/new/statistics/'
decremental_data = np.loadtxt(os.path.join(path_to_decremental,mean_path),delimiter=',')
decremental_std_data = np.loadtxt(os.path.join(path_to_decremental,std_path),delimiter=',')

path_to_incdec = '../runs_trainings/freeze_backbone/incremental_decremental_multilabel/weighted/new/statistics/'
incdec_data = np.loadtxt(os.path.join(path_to_incdec,mean_path),delimiter=',')
incdec_std_data = np.loadtxt(os.path.join(path_to_incdec,std_path),delimiter=',')

path_to_joint = '../runs_trainings/freeze_backbone/joint_incremental_multilabel/weighted/new/statistics/'
joint_data = np.loadtxt(os.path.join(path_to_joint,mean_path),delimiter=',')
joint_std_data = np.loadtxt(os.path.join(path_to_joint,std_path),delimiter=',')

In [41]:
output_path = '../statistics_to_save/subcategories_plots/'
if not os.path.exists(output_path):
    os.mkdir(output_path)
ouput_ap_path = os.path.join(output_path,'ap')
if not os.path.exists(ouput_ap_path):
    os.mkdir(ouput_ap_path)
freeze_path = os.path.join(ouput_ap_path,'freeze')
if not os.path.exists(freeze_path):
    os.mkdir(freeze_path)

for class_name in all_behaviors_dict:
    subcat_names = all_behaviors_dict[class_name]

    baseline_list = []
    decremental_list = []
    incdec_list = []
    joint_list = []
    for idx_subcat in range(len(subcat_names)):
        pos_subcat_in_data = subcategories_names.index(subcat_names[idx_subcat])
        baseline_list.append(baseline_data[pos_subcat_in_data])
        decremental_list.append(decremental_data[pos_subcat_in_data])
        incdec_list.append(incdec_data[pos_subcat_in_data])
        joint_list.append(joint_data[pos_subcat_in_data])
        

    baseline_std_list = []
    decremental_std_list = []
    incdec_std_list = []
    joint_std_list = []
    for idx_subcat in range(len(subcat_names)):
        pos_subcat_in_data = subcategories_names.index(subcat_names[idx_subcat])
        baseline_std_list.append(baseline_std_data[pos_subcat_in_data])
        decremental_std_list.append(decremental_std_data[pos_subcat_in_data])
        incdec_std_list.append(incdec_std_data[pos_subcat_in_data])
        joint_std_list.append(joint_std_data[pos_subcat_in_data])
        

    subcat_means = {
        'Baseline': [baseline_list,baseline_std_list],
        'Decremental': [decremental_list,decremental_std_list],
        'Incdec': [incdec_list,incdec_std_list],
        'Joint': [joint_list,joint_std_list],
        }

    x = np.arange(len(subcat_names))  # the label locations
    width = 0.20  # the width of the bars
    multiplier = 0

    fig, ax = plt.subplots(layout='constrained')

    for attribute, measurements in subcat_means.items():
        offset = width * multiplier
        rects = ax.bar(x + offset, measurements[0], width, label=attribute)
        ax.errorbar(x+offset, measurements[0], measurements[1], fmt='.', color='Black', elinewidth=1,capthick=5,errorevery=1, alpha=0.5, ms=4, capsize = 2)
        #ax.bar_label(rects, padding=3)
        multiplier += 1

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Forgetting on subcat AP')
    ax.set_title('Subcategories Forgetting AP - {}'.format(class_name))
    ax.set_xticks(x + width, subcat_names)
    ax.legend(loc='upper left', ncols=4)
    #ax.set_ylim(-5, 15)
    ax.tick_params(axis='x', labelrotation=90)
    ax.axhline(linewidth=1, color='black')
    fig.savefig(freeze_path + '/class_' + class_name + '.png')
    plt.close(fig)

In [42]:
# recall of subcategories
mean_path = 'mean_over_tasks/forg_recall_per_subcategory.out'
std_path = 'std_over_tasks/forg_recall_per_subcategory.out'

path_to_baseline = '../runs_trainings/freeze_backbone/baseline_multilabel/weighted/new/statistics/'
baseline_data = np.loadtxt(os.path.join(path_to_baseline,mean_path),delimiter=',')
baseline_std_data = np.loadtxt(os.path.join(path_to_baseline,std_path),delimiter=',')

path_to_decremental = '../runs_trainings/freeze_backbone/decremental_multilabel/weighted/new/statistics/'
decremental_data = np.loadtxt(os.path.join(path_to_decremental,mean_path),delimiter=',')
decremental_std_data = np.loadtxt(os.path.join(path_to_decremental,std_path),delimiter=',')

path_to_incdec = '../runs_trainings/freeze_backbone/incremental_decremental_multilabel/weighted/new/statistics/'
incdec_data = np.loadtxt(os.path.join(path_to_incdec,mean_path),delimiter=',')
incdec_std_data = np.loadtxt(os.path.join(path_to_incdec,std_path),delimiter=',')

path_to_joint = '../runs_trainings/freeze_backbone/joint_incremental_multilabel/weighted/new/statistics/'
joint_data = np.loadtxt(os.path.join(path_to_joint,mean_path),delimiter=',')
joint_std_data = np.loadtxt(os.path.join(path_to_joint,std_path),delimiter=',')

In [43]:
# recall
output_path = '../statistics_to_save/subcategories_plots/'
if not os.path.exists(output_path):
    os.mkdir(output_path)
ouput_ap_path = os.path.join(output_path,'recall')
if not os.path.exists(ouput_ap_path):
    os.mkdir(ouput_ap_path)
freeze_path = os.path.join(ouput_ap_path,'freeze')
if not os.path.exists(freeze_path):
    os.mkdir(freeze_path)

for class_name in all_behaviors_dict:
    subcat_names = all_behaviors_dict[class_name]

    baseline_list = []
    decremental_list = []
    incdec_list = []
    joint_list = []
    for idx_subcat in range(len(subcat_names)):
        pos_subcat_in_data = subcategories_names.index(subcat_names[idx_subcat])
        baseline_list.append(baseline_data[pos_subcat_in_data])
        decremental_list.append(decremental_data[pos_subcat_in_data])
        incdec_list.append(incdec_data[pos_subcat_in_data])
        joint_list.append(joint_data[pos_subcat_in_data])
        

    baseline_std_list = []
    decremental_std_list = []
    incdec_std_list = []
    joint_std_list = []
    for idx_subcat in range(len(subcat_names)):
        pos_subcat_in_data = subcategories_names.index(subcat_names[idx_subcat])
        baseline_std_list.append(baseline_std_data[pos_subcat_in_data])
        decremental_std_list.append(decremental_std_data[pos_subcat_in_data])
        incdec_std_list.append(incdec_std_data[pos_subcat_in_data])
        joint_std_list.append(joint_std_data[pos_subcat_in_data])
        

    subcat_means = {
        'Baseline': [baseline_list,baseline_std_list],
        'Decremental': [decremental_list,decremental_std_list],
        'Incdec': [incdec_list,incdec_std_list],
        'Joint': [joint_list,joint_std_list],
        }

    x = np.arange(len(subcat_names))  # the label locations
    width = 0.20  # the width of the bars
    multiplier = 0

    fig, ax = plt.subplots(layout='constrained')

    for attribute, measurements in subcat_means.items():
        offset = width * multiplier
        rects = ax.bar(x + offset, measurements[0], width, label=attribute)
        ax.errorbar(x+offset, measurements[0], measurements[1], fmt='.', color='Black', elinewidth=1,capthick=5,errorevery=1, alpha=0.5, ms=4, capsize = 2)
        #ax.bar_label(rects, padding=3)
        multiplier += 1

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Forgetting on subcat Recall')
    ax.set_title('Subcategories Forgetting Recall - {}'.format(class_name))
    ax.set_xticks(x + width, subcat_names)
    ax.legend(loc='upper left', ncols=4)
    #ax.set_ylim(-5, 15)
    ax.tick_params(axis='x', labelrotation=90)
    ax.axhline(linewidth=1, color='black')
    fig.savefig(freeze_path + '/class_' + class_name + '.png')
    plt.close(fig)

In [44]:
# precision of subcategories
mean_path = 'mean_over_tasks/forg_precision_per_subcategory.out'
std_path = 'std_over_tasks/forg_precision_per_subcategory.out'

path_to_baseline = '../runs_trainings/freeze_backbone/baseline_multilabel/weighted/new/statistics/'
baseline_data = np.loadtxt(os.path.join(path_to_baseline,mean_path),delimiter=',')
baseline_std_data = np.loadtxt(os.path.join(path_to_baseline,std_path),delimiter=',')

path_to_decremental = '../runs_trainings/freeze_backbone/decremental_multilabel/weighted/new/statistics/'
decremental_data = np.loadtxt(os.path.join(path_to_decremental,mean_path),delimiter=',')
decremental_std_data = np.loadtxt(os.path.join(path_to_decremental,std_path),delimiter=',')

path_to_incdec = '../runs_trainings/freeze_backbone/incremental_decremental_multilabel/weighted/new/statistics/'
incdec_data = np.loadtxt(os.path.join(path_to_incdec,mean_path),delimiter=',')
incdec_std_data = np.loadtxt(os.path.join(path_to_incdec,std_path),delimiter=',')

path_to_joint = '../runs_trainings/freeze_backbone/joint_incremental_multilabel/weighted/new/statistics/'
joint_data = np.loadtxt(os.path.join(path_to_joint,mean_path),delimiter=',')
joint_std_data = np.loadtxt(os.path.join(path_to_joint,std_path),delimiter=',')

In [45]:
# precision
output_path = '../statistics_to_save/subcategories_plots/'
if not os.path.exists(output_path):
    os.mkdir(output_path)
ouput_ap_path = os.path.join(output_path,'precision')
if not os.path.exists(ouput_ap_path):
    os.mkdir(ouput_ap_path)
freeze_path = os.path.join(ouput_ap_path,'freeze')
if not os.path.exists(freeze_path):
    os.mkdir(freeze_path)

for class_name in all_behaviors_dict:
    subcat_names = all_behaviors_dict[class_name]

    baseline_list = []
    decremental_list = []
    incdec_list = []
    joint_list = []
    for idx_subcat in range(len(subcat_names)):
        pos_subcat_in_data = subcategories_names.index(subcat_names[idx_subcat])
        baseline_list.append(baseline_data[pos_subcat_in_data])
        decremental_list.append(decremental_data[pos_subcat_in_data])
        incdec_list.append(incdec_data[pos_subcat_in_data])
        joint_list.append(joint_data[pos_subcat_in_data])
        

    baseline_std_list = []
    decremental_std_list = []
    incdec_std_list = []
    joint_std_list = []
    for idx_subcat in range(len(subcat_names)):
        pos_subcat_in_data = subcategories_names.index(subcat_names[idx_subcat])
        baseline_std_list.append(baseline_std_data[pos_subcat_in_data])
        decremental_std_list.append(decremental_std_data[pos_subcat_in_data])
        incdec_std_list.append(incdec_std_data[pos_subcat_in_data])
        joint_std_list.append(joint_std_data[pos_subcat_in_data])
        

    subcat_means = {
        'Baseline': [baseline_list,baseline_std_list],
        'Decremental': [decremental_list,decremental_std_list],
        'Incdec': [incdec_list,incdec_std_list],
        'Joint': [joint_list,joint_std_list],
        }

    x = np.arange(len(subcat_names))  # the label locations
    width = 0.20  # the width of the bars
    multiplier = 0

    fig, ax = plt.subplots(layout='constrained')

    for attribute, measurements in subcat_means.items():
        offset = width * multiplier
        rects = ax.bar(x + offset, measurements[0], width, label=attribute)
        ax.errorbar(x+offset, measurements[0], measurements[1], fmt='.', color='Black', elinewidth=1,capthick=5,errorevery=1, alpha=0.5, ms=4, capsize = 2)
        #ax.bar_label(rects, padding=3)
        multiplier += 1

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Forgetting on subcat Precision')
    ax.set_title('Subcategories Forgetting Precision - {}'.format(class_name))
    ax.set_xticks(x + width, subcat_names)
    ax.legend(loc='upper left', ncols=4)
    #ax.set_ylim(-5, 15)
    ax.tick_params(axis='x', labelrotation=90)
    ax.axhline(linewidth=1, color='black')
    fig.savefig(freeze_path + '/class_' + class_name + '.png')
    plt.close(fig)

In [ ]:
# accuracy of subcategories
mean_path = 'mean_over_tasks/forg_accuracy_per_subcategory.out'
std_path = 'std_over_tasks/forg_accuracy_per_subcategory.out'

path_to_baseline = '../runs_trainings/freeze_backbone/baseline_multilabel/weighted/new/statistics/'
baseline_data = np.loadtxt(os.path.join(path_to_baseline,mean_path),delimiter=',')
baseline_std_data = np.loadtxt(os.path.join(path_to_baseline,std_path),delimiter=',')

path_to_decremental = '../runs_trainings/freeze_backbone/decremental_multilabel/weighted/new/statistics/'
decremental_data = np.loadtxt(os.path.join(path_to_decremental,mean_path),delimiter=',')
decremental_std_data = np.loadtxt(os.path.join(path_to_decremental,std_path),delimiter=',')

path_to_incdec = '../runs_trainings/freeze_backbone/incremental_decremental_multilabel/weighted/new/statistics/'
incdec_data = np.loadtxt(os.path.join(path_to_incdec,mean_path),delimiter=',')
incdec_std_data = np.loadtxt(os.path.join(path_to_incdec,std_path),delimiter=',')

path_to_joint = '../runs_trainings/freeze_backbone/joint_incremental_multilabel/weighted/new/statistics/'
joint_data = np.loadtxt(os.path.join(path_to_joint,mean_path),delimiter=',')
joint_std_data = np.loadtxt(os.path.join(path_to_joint,std_path),delimiter=',')

In [ ]:
# precision
output_path = '../statistics_to_save/subcategories_plots/'
if not os.path.exists(output_path):
    os.mkdir(output_path)
ouput_ap_path = os.path.join(output_path,'accuracy')
if not os.path.exists(ouput_ap_path):
    os.mkdir(ouput_ap_path)
freeze_path = os.path.join(ouput_ap_path,'freeze')
if not os.path.exists(freeze_path):
    os.mkdir(freeze_path)

for class_name in all_behaviors_dict:
    subcat_names = all_behaviors_dict[class_name]

    baseline_list = []
    decremental_list = []
    incdec_list = []
    joint_list = []
    for idx_subcat in range(len(subcat_names)):
        pos_subcat_in_data = subcategories_names.index(subcat_names[idx_subcat])
        baseline_list.append(baseline_data[pos_subcat_in_data])
        decremental_list.append(decremental_data[pos_subcat_in_data])
        incdec_list.append(incdec_data[pos_subcat_in_data])
        joint_list.append(joint_data[pos_subcat_in_data])
        

    baseline_std_list = []
    decremental_std_list = []
    incdec_std_list = []
    joint_std_list = []
    for idx_subcat in range(len(subcat_names)):
        pos_subcat_in_data = subcategories_names.index(subcat_names[idx_subcat])
        baseline_std_list.append(baseline_std_data[pos_subcat_in_data])
        decremental_std_list.append(decremental_std_data[pos_subcat_in_data])
        incdec_std_list.append(incdec_std_data[pos_subcat_in_data])
        joint_std_list.append(joint_std_data[pos_subcat_in_data])
        

    subcat_means = {
        'Baseline': [baseline_list,baseline_std_list],
        'Decremental': [decremental_list,decremental_std_list],
        'Incdec': [incdec_list,incdec_std_list],
        'Joint': [joint_list,joint_std_list],
        }

    x = np.arange(len(subcat_names))  # the label locations
    width = 0.20  # the width of the bars
    multiplier = 0

    fig, ax = plt.subplots(layout='constrained')

    for attribute, measurements in subcat_means.items():
        offset = width * multiplier
        rects = ax.bar(x + offset, measurements[0], width, label=attribute)
        ax.errorbar(x+offset, measurements[0], measurements[1], fmt='.', color='Black', elinewidth=1,capthick=5,errorevery=1, alpha=0.5, ms=4, capsize = 2)
        #ax.bar_label(rects, padding=3)
        multiplier += 1

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Forgetting on subcat Accuracy')
    ax.set_title('Subcategories Forgetting Accuracy - {}'.format(class_name))
    ax.set_xticks(x + width, subcat_names)
    ax.legend(loc='upper left', ncols=4)
    #ax.set_ylim(-5, 15)
    ax.tick_params(axis='x', labelrotation=90)
    ax.axhline(linewidth=1, color='black')
    fig.savefig(freeze_path + '/class_' + class_name + '.png')
    plt.close(fig)